# PARTE 1: Preliminares

In [1]:
import re
import random
import numpy as np
import datetime
import gc

In [2]:
# lê o corpus usando utf-8
with open(r'CETENFolha-1.0_jan2014.cg',encoding='utf-8') as f:
    corpus=f.read()
corpus=re.sub('<s frag>','<s>',corpus)

In [3]:
print(datetime.datetime.now())
sentencas=re.split('<s>',corpus) #separa as sentenças
sentencas_deletadas=[] #armazena todas as sentenças que serão deletadas, apenas para conferir se nada importante está sendo jogado fora
for i in reversed(range(len(sentencas))): #itera nas sentenças
    if i%100000==0:
        print(i)
    sentencas[i]=sentencas[i].split('\n') #separa a sentença em linhas
    sentencas[i]=[linha for linha in sentencas[i] if (len(linha)!=0 and linha[0]!='<')] #exclue as linhas vazias ou que começam com '<'
    if len(sentencas[i])==0: #se a sentença for vazia, é retirada
        del sentencas[i]
        continue
    for j in reversed(range(len(sentencas[i]))): 
        try:
            if re.search('^\$\S',sentencas[i][j]) and not re.search('^\$\([0-9-]+\)=',sentencas[i][j]): #caso seja pontuaçao, mas não seja número de telefone
                token=re.findall('^\$(\S+)',sentencas[i][j])[0].lower()
                if token in ['!','?','.',':']:
                    pos='.'
                else:
                    pos=','
                sentencas[i][j]=(token,pos)#,sentencas[i][j])
            else: #caso seja palavra ou número de telefone
                sentencas[i][j]=sentencas[i][j].replace(' Pr ',' PR ').replace(' Iremp ',' IREMP ')
                token=re.findall('^\S+',sentencas[i][j])[0].lower()
                pos=sentencas[i][j]
                pos=re.sub('^\S+','',pos)
                pos=re.sub('[@#].*$','',pos)
                pos=re.sub('<[\S]+','',pos)
                pos=re.sub('[\S]+>','',pos)
                pos=re.sub('\[.*?\]','',pos)
                pos=re.sub('-[a-zØ-öø-ÿ]+','',pos)
                pos=re.sub('[a-zØ-öø-ÿ]+-','',pos)
                pos=re.sub('\t','',pos)
                pos=re.sub('[a-z]','',pos)
                pos=re.sub('GER','',pos)
                pos=re.sub('PRON','N',pos)
                pos=pos.strip()
                pos=pos.split()[0]
                if pos=='M':
                    raise Exception('pos==M')
                if pos=='[':
                    raise Exception('pos==[')
                if pos=='PRP_DET':
                    raise Exception('pos==PRP_DET')
                if pos=='PP':
                    raise Exception('pos==PP')
                sentencas[i][j]=(token,pos)#,sentencas[i][j])
            
        except:
            if 'sem=' in sentencas[i][j]: #deleta somente o token da sentença
                del sentencas[i][j]
            else: #deleta a sentença inteira
                sentencas_deletadas.append(sentencas[i])
                del sentencas[i]
                break
print(datetime.datetime.now())

2018-10-08 02:51:07.338028
1500000
1400000
1300000
1200000
1100000
1000000
900000
800000
700000
600000
500000
400000
300000
200000
100000
0
2018-10-08 03:02:02.196038


In [4]:
corpus=''
gc.collect()

0

In [5]:
dict_freq={}
full_pos=set({})
for i in range(len(sentencas)):
    if i%100000==0:
        print(i)
    for j in range(len(sentencas[i])):
        if sentencas[i][j][0] not in dict_freq:
            dict_freq[sentencas[i][j][0]]={}
        try:
            dict_freq[sentencas[i][j][0]][sentencas[i][j][1]]=dict_freq[sentencas[i][j][0]].get(sentencas[i][j][1],0)+1
            full_pos.add(sentencas[i][j][1])
        except:
            print(i,j)
            print(sentencas[i][j])


0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000


In [6]:
for token in dict_freq:
    max_pos = max(dict_freq[token], key=dict_freq[token].get)
    dict_freq[token]['max_pos']=max_pos

In [7]:
total=0
corretos=0
for i in range(len(sentencas)):
    if i%100000==0:
        print(i)
    for j in range(len(sentencas[i])):
        total+=1
#         sentencas[i][j]=sentencas[i][j]+tuple([dict_freq[sentencas[i][j][0]]['max_pos']])
#         if sentencas[i][j][1]==sentencas[i][j][4]:
        if sentencas[i][j][1]==dict_freq[sentencas[i][j][0]]['max_pos']:
            corretos+=1

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000


In [8]:
full_pos

{',',
 '.',
 'ADJ',
 'ADV',
 'DET',
 'EC',
 'IN',
 'KC',
 'KS',
 'N',
 'NUM',
 'PERS',
 'PROP',
 'PRP',
 'SPEC',
 'V'}

In [9]:
len(full_pos)

16

In [10]:
total

27565496

In [11]:
corretos

26336323

In [12]:
corretos/total

0.9554090011658053

In [13]:
print(datetime.datetime.now())

2018-10-08 03:03:10.944182


# PARTE 2: Etiquetagem por Contagem

In [14]:
print(datetime.datetime.now())

2018-10-08 03:03:10.962194


In [15]:
sentencas_shuffle=list(sentencas)

In [16]:
random.shuffle(sentencas_shuffle)

In [17]:
n_treino=round(len(sentencas_shuffle)*0.8)

In [18]:
sentencas_treino=sentencas_shuffle[:n_treino]

In [19]:
sentencas_teste=sentencas_shuffle[n_treino:]

In [47]:
len(sentencas_treino)

1276098

In [20]:
colunas=sorted(list(full_pos)+['INICIO','FIM'])
linhas=[()]
HMM_order=2
for i in range(HMM_order):
    for j in reversed(range(len(linhas))):
        for k in reversed(range(len(colunas))):
            linhas.append(linhas[j]+(colunas[k],))
        del linhas[j]


In [21]:
A=np.zeros((len(linhas),len(colunas)),np.int)

B={}
for i in range(len(sentencas_treino)):
    if i%100000==0:
        print(i)
    sent=[(None,'INICIO',None,None,None)]*2+sentencas_treino[i]+[('%%%%%','FIM',None,None,None)]
    for j in range(2,len(sent)):
        A[linhas.index((sent[j-2][1],sent[j-1][1])),colunas.index(sent[j][1])]+=1
        if sent[j][1] not in B:
            B[sent[j][1]]={}
        B[sent[j][1]][sent[j][0]]=B[sent[j][1]].get(sent[j][0],0)+1
A=A+np.ones(A.shape)/10
A=A/A.sum(axis=1).reshape(len(linhas),1)
A=np.log(A)
for key in B:
    B[key]['#####']=0.1
    soma=sum(B[key].values())
    for keykey in B[key]:
        B[key][keykey]=np.log(B[key][keykey]/soma)

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000


## Daqui para frente roda o Viterbi de 10k em 10k sentenças

In [22]:
# for i in range(len(sentencas_teste)):
print(datetime.datetime.now())
cont=0
testes_classificados=[]
for i in range(10000):
    if i%100==0:
        print(i)
    sent=sentencas_teste[i]+[('%%%%%','FIM',None,None,None)]
    viterbi={}
    viterbi[('INICIO','INICIO')]=[('INICIO',0),('INICIO',0)]
    
    for j in range(len(sent)):
        viterbi_old=dict(viterbi)
        viterbi={}
        for key in viterbi_old:
            for l in range(len(colunas)):
                if colunas[l]=='INICIO':
                    continue
                emissao=max(B[colunas[l]].get(sent[j][0],np.NINF),B[colunas[l]]['#####'])
                log_prob=viterbi_old[key][-1][1]+A[linhas.index(key),l]+emissao
                if viterbi.get((key[1],colunas[l]),[(None,np.NINF)])[-1][1]<log_prob:
                    viterbi[(key[1],colunas[l])]=viterbi_old[key]+[(colunas[l],log_prob)]
    testes_classificados.append(max([viterbi[key] for key in viterbi],key=lambda x:x[-1][1])[2:])
print(datetime.datetime.now())

2018-10-08 03:06:43.862926
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
2018-10-08 05:03:45.817867


In [23]:
# for i in range(len(sentencas_teste)):
print(datetime.datetime.now())
cont=0
# testes_classificados=[]
for i in range(10000,20000):
    if i%100==0:
        print(i)
    sent=sentencas_teste[i]+[('%%%%%','FIM',None,None,None)]
    viterbi={}
    viterbi[('INICIO','INICIO')]=[('INICIO',0),('INICIO',0)]
    
    for j in range(len(sent)):
        viterbi_old=dict(viterbi)
        viterbi={}
        for key in viterbi_old:
            for l in range(len(colunas)):
                if colunas[l]=='INICIO':
                    continue
                emissao=max(B[colunas[l]].get(sent[j][0],np.NINF),B[colunas[l]]['#####'])
                log_prob=viterbi_old[key][-1][1]+A[linhas.index(key),l]+emissao
                if viterbi.get((key[1],colunas[l]),[(None,np.NINF)])[-1][1]<log_prob:
                    viterbi[(key[1],colunas[l])]=viterbi_old[key]+[(colunas[l],log_prob)]
    testes_classificados.append(max([viterbi[key] for key in viterbi],key=lambda x:x[-1][1])[2:])
print(datetime.datetime.now())

2018-10-08 05:03:45.932152
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18500
18600
18700
18800
18900
19000
19100
19200
19300
19400
19500
19600
19700
19800
19900
2018-10-08 06:59:12.094964


In [24]:
# for i in range(len(sentencas_teste)):
print(datetime.datetime.now())
cont=0
# testes_classificados=[]
for i in range(20000,30000):
    if i%100==0:
        print(i)
    sent=sentencas_teste[i]+[('%%%%%','FIM',None,None,None)]
    viterbi={}
    viterbi[('INICIO','INICIO')]=[('INICIO',0),('INICIO',0)]
    
    for j in range(len(sent)):
        viterbi_old=dict(viterbi)
        viterbi={}
        for key in viterbi_old:
            for l in range(len(colunas)):
                if colunas[l]=='INICIO':
                    continue
                emissao=max(B[colunas[l]].get(sent[j][0],np.NINF),B[colunas[l]]['#####'])
                log_prob=viterbi_old[key][-1][1]+A[linhas.index(key),l]+emissao
                if viterbi.get((key[1],colunas[l]),[(None,np.NINF)])[-1][1]<log_prob:
                    viterbi[(key[1],colunas[l])]=viterbi_old[key]+[(colunas[l],log_prob)]
    testes_classificados.append(max([viterbi[key] for key in viterbi],key=lambda x:x[-1][1])[2:])
print(datetime.datetime.now())

2018-10-08 06:59:12.136573
20000
20100
20200
20300
20400
20500
20600
20700
20800
20900
21000
21100
21200
21300
21400
21500
21600
21700
21800
21900
22000
22100
22200
22300
22400
22500
22600
22700
22800
22900
23000
23100
23200
23300
23400
23500
23600
23700
23800
23900
24000
24100
24200
24300
24400
24500
24600
24700
24800
24900
25000
25100
25200
25300
25400
25500
25600
25700
25800
25900
26000
26100
26200
26300
26400
26500
26600
26700
26800
26900
27000
27100
27200
27300
27400
27500
27600
27700
27800
27900
28000
28100
28200
28300
28400
28500
28600
28700
28800
28900
29000
29100
29200
29300
29400
29500
29600
29700
29800
29900
2018-10-08 08:52:55.018377


In [25]:
# for i in range(len(sentencas_teste)):
print(datetime.datetime.now())
cont=0
# testes_classificados=[]
for i in range(30000,40000):
    if i%100==0:
        print(i)
    sent=sentencas_teste[i]+[('%%%%%','FIM',None,None,None)]
    viterbi={}
    viterbi[('INICIO','INICIO')]=[('INICIO',0),('INICIO',0)]
    
    for j in range(len(sent)):
        viterbi_old=dict(viterbi)
        viterbi={}
        for key in viterbi_old:
            for l in range(len(colunas)):
                if colunas[l]=='INICIO':
                    continue
                emissao=max(B[colunas[l]].get(sent[j][0],np.NINF),B[colunas[l]]['#####'])
                log_prob=viterbi_old[key][-1][1]+A[linhas.index(key),l]+emissao
                if viterbi.get((key[1],colunas[l]),[(None,np.NINF)])[-1][1]<log_prob:
                    viterbi[(key[1],colunas[l])]=viterbi_old[key]+[(colunas[l],log_prob)]
    testes_classificados.append(max([viterbi[key] for key in viterbi],key=lambda x:x[-1][1])[2:])
print(datetime.datetime.now())

2018-10-08 10:08:29.879777
30000
30100
30200
30300
30400
30500
30600
30700
30800
30900
31000
31100
31200
31300
31400
31500
31600
31700
31800
31900
32000
32100
32200
32300
32400
32500
32600
32700
32800
32900
33000
33100
33200
33300
33400
33500
33600
33700
33800
33900
34000
34100
34200
34300
34400
34500
34600
34700
34800
34900
35000
35100
35200
35300
35400
35500
35600
35700
35800
35900
36000
36100
36200
36300
36400
36500
36600
36700
36800
36900
37000
37100
37200
37300
37400
37500
37600
37700
37800
37900
38000
38100
38200
38300
38400
38500
38600
38700
38800
38900
39000
39100
39200
39300
39400
39500
39600
39700
39800
39900
2018-10-08 12:04:08.334056


In [26]:
# for i in range(len(sentencas_teste)):
print(datetime.datetime.now())
cont=0
# testes_classificados=[]
for i in range(40000,50000):
    if i%100==0:
        print(i)
    sent=sentencas_teste[i]+[('%%%%%','FIM',None,None,None)]
    viterbi={}
    viterbi[('INICIO','INICIO')]=[('INICIO',0),('INICIO',0)]
    
    for j in range(len(sent)):
        viterbi_old=dict(viterbi)
        viterbi={}
        for key in viterbi_old:
            for l in range(len(colunas)):
                if colunas[l]=='INICIO':
                    continue
                emissao=max(B[colunas[l]].get(sent[j][0],np.NINF),B[colunas[l]]['#####'])
                log_prob=viterbi_old[key][-1][1]+A[linhas.index(key),l]+emissao
                if viterbi.get((key[1],colunas[l]),[(None,np.NINF)])[-1][1]<log_prob:
                    viterbi[(key[1],colunas[l])]=viterbi_old[key]+[(colunas[l],log_prob)]
    testes_classificados.append(max([viterbi[key] for key in viterbi],key=lambda x:x[-1][1])[2:])
print(datetime.datetime.now())

2018-10-08 13:28:24.127391
40000
40100
40200
40300
40400
40500
40600
40700
40800
40900
41000
41100
41200
41300
41400
41500
41600
41700
41800
41900
42000
42100
42200
42300
42400
42500
42600
42700
42800
42900
43000
43100
43200
43300
43400
43500
43600
43700
43800
43900
44000
44100
44200
44300
44400
44500
44600
44700
44800
44900
45000
45100
45200
45300
45400
45500
45600
45700
45800
45900
46000
46100
46200
46300
46400
46500
46600
46700
46800
46900
47000
47100
47200
47300
47400
47500
47600
47700
47800
47900
48000
48100
48200
48300
48400
48500
48600
48700
48800
48900
49000
49100
49200
49300
49400
49500
49600
49700
49800
49900
2018-10-08 15:42:09.786142


In [33]:
# for i in range(len(sentencas_teste)):
print(datetime.datetime.now())
cont=0
# testes_classificados=[]
for i in range(50000,60000):
    if i%100==0:
        print(i)
    sent=sentencas_teste[i]+[('%%%%%','FIM',None,None,None)]
    viterbi={}
    viterbi[('INICIO','INICIO')]=[('INICIO',0),('INICIO',0)]
    
    for j in range(len(sent)):
        viterbi_old=dict(viterbi)
        viterbi={}
        for key in viterbi_old:
            for l in range(len(colunas)):
                if colunas[l]=='INICIO':
                    continue
                emissao=max(B[colunas[l]].get(sent[j][0],np.NINF),B[colunas[l]]['#####'])
                log_prob=viterbi_old[key][-1][1]+A[linhas.index(key),l]+emissao
                if viterbi.get((key[1],colunas[l]),[(None,np.NINF)])[-1][1]<log_prob:
                    viterbi[(key[1],colunas[l])]=viterbi_old[key]+[(colunas[l],log_prob)]
    testes_classificados.append(max([viterbi[key] for key in viterbi],key=lambda x:x[-1][1])[2:])
print(datetime.datetime.now())

2018-10-08 17:02:10.716220
50000
50100
50200
50300
50400
50500
50600
50700
50800
50900
51000
51100
51200
51300
51400
51500
51600
51700
51800
51900
52000
52100
52200
52300
52400
52500
52600
52700
52800
52900
53000
53100
53200
53300
53400
53500
53600
53700
53800
53900
54000
54100
54200
54300
54400
54500
54600
54700
54800
54900
55000
55100
55200
55300
55400
55500
55600
55700
55800
55900
56000
56100
56200
56300
56400
56500
56600
56700
56800
56900
57000
57100
57200
57300
57400
57500
57600
57700
57800
57900
58000
58100
58200
58300
58400
58500
58600
58700
58800
58900
59000
59100
59200
59300
59400
59500
59600
59700
59800
59900
2018-10-08 19:17:10.953139


In [27]:
list(zip([e[1] for e in sentencas_teste[0]],testes_classificados[0]))

[('DET', ('DET', -2.3613424198220643)),
 ('N', ('N', -10.367096364920029)),
 ('V', ('V', -18.932216697471848)),
 ('PRP', ('PRP', -22.693054875433923)),
 ('V', ('V', -32.864302366191325)),
 ('ADV', ('ADV', -39.61555497600045)),
 ('DET', ('DET', -42.45546106540909)),
 ('PROP', ('PROP', -51.461848375567214)),
 ('ADJ', ('ADJ', -65.72154617660699)),
 ('PRP', ('PRP', -67.85094530847785)),
 ('N', ('N', -77.49720331989587)),
 ('KC', ('KC', -80.33317072629131)),
 ('N', ('N', -89.07405278019245)),
 ('.', ('.', -90.99051350403263))]

In [109]:
corretos_viterbi=0
total_viterbi=0
for i in range(60000):
    for j in range(len(sentencas_teste[i])):
        if sentencas_teste[i][j][1]==testes_classificados[i][j][0]:
            corretos_viterbi+=1
        total_viterbi+=1

In [110]:
total_viterbi

1038305

In [111]:
corretos_viterbi

1006324

In [112]:
corretos_viterbi/total_viterbi

0.9691988384915801

In [113]:
dict_freq={}
full_pos=set({})
for i in range(len(sentencas_treino)):
    if i%100000==0:
        print(i)
    for j in range(len(sentencas_treino[i])):
        if sentencas_treino[i][j][0] not in dict_freq:
            dict_freq[sentencas_treino[i][j][0]]={}
        try:
            dict_freq[sentencas_treino[i][j][0]][sentencas_treino[i][j][1]]=dict_freq[sentencas_treino[i][j][0]].get(sentencas_treino[i][j][1],0)+1
            full_pos.add(sentencas_treino[i][j][1])
        except:
            print(i,j)
            print(sentencas_treino[i][j])


0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000


In [114]:
for token in dict_freq:
    max_pos = max(dict_freq[token], key=dict_freq[token].get)
    dict_freq[token]['max_pos']=max_pos

In [115]:
total_naive=0
corretos_naive=0
for i in range(60000):
    if i%100000==0:
        print(i)
    for j in range(len(sentencas_teste[i])):
#         if sentencas[i][j][3]!='pontuação':
#             continue
        total_naive+=1
#         try:
#             sentencas_teste[i][j]=sentencas_teste[i][j]+tuple([dict_freq[sentencas_teste[i][j][0]]['max_pos']])
        try:
            if sentencas_teste[i][j][1]==dict_freq[sentencas_teste[i][j][0]]['max_pos']:
                corretos_naive+=1
        except:
            pass
#         except:
#             pass

0


In [116]:
total_naive

1038305

In [117]:
corretos_naive

981532

In [118]:
corretos_naive/total_naive

0.9453214614202956